# Student Course Report Analysis


In [89]:
# Imports
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import functools

from os.path import join, dirname
from pymongo import MongoClient
from pprint import pprint
from dotenv import load_dotenv
from IPython.display import display, JSON, HTML

# Load environment variables
load_dotenv(join(os.path.abspath(""), "../.env.local"))

pass

In [90]:
# Connect to database
mongo_client = MongoClient(os.environ.get("MONGO_URI"))
db = mongo_client.george

# Load data
professors = pd.DataFrame(db["professors"].find({}))
courses = pd.DataFrame(db["courses"].find({}))
sections = pd.DataFrame(db["sections"].find({}))

In [91]:
# The question text for each number
question_texts = {
    1: "My overall rating of the quality of this course is",
    2: "My overall rating of the instructor's teaching is",
    3: "The educational value of the assigned work was",
    4: "The instructor's organization of the course was",
    5: "The instructor's clarity in communicating course objectives was",
    6: "The instructor's skill in providing understandable explanations was",
    7: "The amount I learned from the course was",
    8: "The intellectual challenge presented by the course was",
    9: "The instructor's personal interest in helping students learn was",
    10: "The instructor stimulated my interest in the subject matter",
    11: "The amount of reading, homework, and other assigned work was",
    12: "The instructor was well prepared to teach class.",
    13: "The instructor encouraged students to ask questions.",
    14: "The instructor treated students with respect.",
    15: "Instructor feedback on exams/assignments was timely and helpful.",
    16: "The exams and/or evaluations were good measures of the material covered.",
    17: "My grades were determined in a fair and impartial manner.",
    18: "What grade do you think you will receive in this course?",
    19: "On average, what were the total hours spent in each 7-day week OUTSIDE of formally scheduled class time in work related to this course (including studying, reading, writing, homework, rehearsal, etc.)?"
}

# Categories of questions (subjective)
question_categories = {
    "course_quality": [1, 2, 3, 4, 5, 6],
    "course_difficulty": [7, 8, 9, 10, 11, 19],
    "instructor_quality": [12, 13, 14, 15, 16],
    "grades": [17, 18]
}

In [95]:
# Calculate the average response of every question in a category
def add_category(row, questions):
    category_sum = 0
    for question in row.report:
        if question["questionNumber"] not in questions:
            continue
        #print(question["questionNumber"])
        responses = question["responses"]
        weighted_responses = [x*(i+1) for i, x in enumerate(responses)]

        if sum(responses) == 0:
            return None
        else:
            mean_response = sum(weighted_responses) / sum(responses)

        category_sum += mean_response
        #print(responses, weighted_responses, mean_response)
    
    category_mean = category_sum / len(questions)
    
    return category_mean

for category, questions in question_categories.items():
    sections[category] = sections.apply(lambda row: add_category(row, questions), axis=1)


display(sections)

,_id,professorId,professorName,courseId,courseName,term,sectionType,report,course_quality,course_difficulty,instructor_quality,grades
0,6245a3a80008c19b8aeb6e90,811311,"Brahimi, Mohamed",AB 1531,ELEMENTARY ARABIC I,201901_A,Lecture,"[{'questionNumber': 1, 'responses': [0, 0, 1, ...",4.259259,3.435185,4.500000,3.333333
1,6245a3a80008c19b8aeb6e91,1005062,"El Hamzaoui, Mohammed",AB 1531,ELEMENTARY ARABIC I,201901_A,Lecture,"[{'questionNumber': 1, 'responses': [0, 1, 4, ...",3.852941,3.637255,4.129412,3.088235
2,6245a3a90008c19b8aeb6e92,811311,"Brahimi, Mohamed",AB 1532,ELEMENTARY ARABIC II,201901_B,Lecture,"[{'questionNumber': 1, 'responses': [0, 0, 2, ...",4.431917,3.678649,4.711111,3.459150
3,6245a3a90008c19b8aeb6e93,1005062,"El Hamzaoui, Mohammed",AB 1533,ELEMENTARY ARABIC III,201902_C,Lecture,"[{'questionNumber': 1, 'responses': [0, 0, 1, ...",4.407407,3.585648,4.533333,3.333333
4,6245a3aa0008c19b8aeb6e94,811311,"Brahimi, Mohamed",AB 210X,INTERMEDIATE ARABIC I,201902_D,Lecture,"[{'questionNumber': 1, 'responses': [0, 0, 1, ...",4.145833,3.500000,4.475000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
7536,6245b41c0008c19b8aeb8c00,340119884,"Runstrom, Scott",WR 3210,Technical Writing,202201_A,Lecture,"[{'questionNumber': 1, 'responses': [0, 0, 0, ...",4.833333,3.659259,4.900000,3.200000
7537,6245b41d0008c19b8aeb8c01,786726241,"Lewis, Kevin",WR 3210,Technical Writing,202202_C,Lecture,"[{'questionNumber': 1, 'responses': [0, 0, 0, ...",4.719048,3.769231,4.800000,3.433333
7538,6245b41d0008c19b8aeb8c02,601204077,"Madan, Ryan",WR 4111,Research Methods In Writing,202202_C,Lecture,"[{'questionNumber': 1, 'responses': [0, 0, 0, ...",4.333333,3.555556,4.466667,3.333333
7539,6245b41e0008c19b8aeb8c03,638496388,"Faber, Brenton",WR 4210,- Medical Writing,202201_B,Lecture,"[{'questionNumber': 1, 'responses': [0, 0, 1, ...",4.833333,4.166667,5.000000,3.000000
